
Vedlegg tilgjengelige fram til 26. nov. 2019
 
Hei!

Nå er jeg ikke så veldig langt unna med å ha ferdig analysen av de forrige resultatene jeg fikk. Det er ganske gøy å holde på med det. Men jeg jobber på spreng for å holde tidsskjemaet mitt :)  


Dette betyr at jeg snart gjerne vil få hentet ut et siste sett med resultater. Som vi snakket om sist, da ønsker jeg å gjøre bruk av flere automatisk genererte temaer. Jeg satser på å sende deg nye temaer innen torsdag. Det vil bli mindre omfattende enn sist. 



Men kunne du i mellomtiden hjelpe meg litt med “dyp-topic-modellering” og legge inn flere måter å generere temaer på i notebøkene av de kvinnelige forfatterne mine? 


Hertha- notebooken er genial i så måte. Her har du lagt inn mange forskjellige parametere for tema-generering.   
Klikk for å laste ned
TM_Hertha.ipynb
385 kB


Kunne du sett litt nærmere på de andre notebookene også? Er det flere nyttige fremgangsmåter for automatisk tema-generering du kan legge til? 

Klikk for å laste ned
TM_Til Sverriges Qvinder .ipynb
13 kB
 
Klikk for å laste ned
denconstitutionelle.txt
14 kB
 
Klikk for å laste ned
Amtmandens døtre .ipynb
1,4 MB
 
Klikk for å laste ned
Indiana.ipynb
3,1 MB
 
Klikk for å laste ned
sand-indiana.txt
570 kB


I Amtmandens Døtre-notebooken har vi brukt denne teksten  
Klikk for å laste ned
ad3.txt
667 kB
 men det ville vært veldig fint om vi heller kunne brukt 1855-utgaven: 
https://www.nb.no/items/fbf5c6432c86426817fe167282f603ad?page=519&searchText=Amtmandens%20døttre


Til slutt skulle jeg veldig gjerne laget meg et dokument av de tre artiklene til Aasta Hansteen også. For jeg vil gjerne også få generert automatiske temaer fra hennes tekster. Er det mulig å ta ut Dagbladet-utgavene og legge den inn i et txt-dokument? Jeg kan i tilfelle gå inn i det etterpå og fjerne all annen tekst fra de respektive avisene (slik jeg har gjort for “Til Sverriges Qvinder”) . Her er de tre Dagblad-utgavene det gjelder: 

Klikk for å laste ned
Hansteen 12 August 1870.pdf
19,9 MB
 
Klikk for å laste ned
Hansteen 06 August 1870.pdf
21,4 MB
 
Klikk for å laste ned
Hansteen 13 July 1870.pdf
20,6 MB


In [1]:
import requests
import os
from IPython.display import HTML, Markdown, display
from urllib.parse import urlparse, urljoin
import ipywidgets as widgets

def code_toggle(button_text = "Klikk for å vise/skjule kodeceller"):
    from IPython.display import HTML, display

    display(
        HTML(
        '''<div>
                <style>
                 .mybutton {
                    background-color: lightgrey;
                    border: none;
                    color: white;
                    padding: 10px 16px;
                    text-align: center;
                    text-decoration: none;
                    display: inline-block;
                    font-size: 16px;
                    margin: 4px 2px;
                    cursor: pointer;
                }
            </style>
            <script>
                code_show=true; 
                function code_toggle() {
                 if (code_show){
                 $('div.input').hide();
                 } else {
                 $('div.input').show();
                 }
                 code_show = !code_show
                } 
                $( document ).ready(code_toggle);
            </script>
            <form  action="javascript:code_toggle()">
                <input class='mybutton' type="submit" value=''' + '"'  + button_text + '"' + '''>
            </form>
        </div>'''
        ))
    
def printmd(S):
    display(Markdown(S))
    return

def update(module="", overwrite=True, silent=False):
    """Fetch modules from Github and write them to folder"""
    nba = requests.get(
        "https://raw.githubusercontent.com/Yoonsen/Modules/master/{module}.py".format(module=module),
        headers={'Cache-Control': 'no-cache'}
        )
    filename = '{m}.py'.format(m=module)
    if nba.status_code == 200:
        file_exists = os.path.exists(filename)
        if file_exists and not(overwrite):
            if not silent:
                printmd("File {f} exists - call `update('{m}', overwrite = True)` in order to download module `{m}` anyway".format(f = os.path.abspath(filename), m = module))
        else:
            nba = nba.text
            with open(filename,'w', encoding='UTF-8') as pyfile:
                pyfile.write(nba)
                pyfile.flush()
                pyfile.close()
            if not silent:
                printmd("Updated file `{module}.py`".format(module= os.path.abspath(module)))
    else:
        printmd( 
            """{intro} for {module} with error {code}""".format(
                intro = "An error occured during download", 
                module = module, 
                code= nba.status_code
            )
        )
    return



In [28]:
import pandas as pd
from dhlab.nbtokenizer import tokenize
import dhlab.graph_networkx_louvain as gnl
from IPython.display import HTML, Markdown, display
from collections import Counter
import networkx as nx
from bs4 import BeautifulSoup
import json

In [20]:
filstruct = os.walk('hertha-txt')
myfiles = [os.path.join('hertha-txt', f) for _ ,_ , file in filstruct for f in file if f.startswith('kap') and f.endswith('.html')]
myfiles

['hertha-txt\\kap01.html',
 'hertha-txt\\kap02.html',
 'hertha-txt\\kap03.html',
 'hertha-txt\\kap04.html',
 'hertha-txt\\kap05.html',
 'hertha-txt\\kap06.html',
 'hertha-txt\\kap07.html',
 'hertha-txt\\kap08.html',
 'hertha-txt\\kap09.html',
 'hertha-txt\\kap10.html',
 'hertha-txt\\kap11.html',
 'hertha-txt\\kap12.html',
 'hertha-txt\\kap13.html',
 'hertha-txt\\kap14.html',
 'hertha-txt\\kap15.html',
 'hertha-txt\\kap16.html',
 'hertha-txt\\kap17.html',
 'hertha-txt\\kap18.html',
 'hertha-txt\\kap19.html',
 'hertha-txt\\kap20.html',
 'hertha-txt\\kap21.html',
 'hertha-txt\\kap22.html',
 'hertha-txt\\kap23.html',
 'hertha-txt\\kap24.html',
 'hertha-txt\\kap25.html',
 'hertha-txt\\kap26.html',
 'hertha-txt\\kap27.html',
 'hertha-txt\\kap28.html',
 'hertha-txt\\kap29.html',
 'hertha-txt\\kap30.html',
 'hertha-txt\\kap31.html',
 'hertha-txt\\kap32.html',
 'hertha-txt\\kap33.html',
 'hertha-txt\\kap34.html',
 'hertha-txt\\kap35.html',
 'hertha-txt\\kap36.html',
 'hertha-txt\\kap37.html',
 

# Extract text from file

In [31]:
text = []
for filname in myfiles:
    with open(filname, encoding='utf-8') as f:
        soup = BeautifulSoup(f.read(), 'html')
        for n in soup.find_all('p'):
            text.append(tokenize(n.text))

In [33]:
#temaene ble lagret med annen notebook - Heidis definisjoner_0_november
with open('../Nye tema november 2019/heidis_temadefinisjoner_november.json') as f:
    first = json.load(f)

wordbags = dict()
for f in first:
    for wb in first[f]:
        wordbags[wb] = first[f][wb]

In [91]:
def paragraphs(textfile):
    with open(textfile, encoding='utf-8') as f:
        text = f.read()
    # return tokenizes paragraphs 
    return [tokenize(p) for p in text.split('\n')]

In [94]:
def paragraphs_hot(textfile, freqlim = 1):
    # return tokenizes paragraphs 
    i = 0
    struct = dict()
    for p in paragraphs(textfile):
        struct[i] = Counter(p)
        i += 1
    return pd.DataFrame(struct).fillna(0)

def paragraphs_hot_from_list(paragraphs, freqlim = 1):
    # return tokenizes paragraphs 
    i = 0
    struct = dict()
    for p in paragraphs:
        struct[i] = Counter(p)
        i += 1
    return pd.DataFrame(struct).fillna(0)
        

In [95]:
texthot = paragraphs_hot_from_list(text)

In [99]:
freqs = texthot.sum(axis=1).sort_values()

In [40]:
def pcoll(word, paras):
    """paras as a numbered dictionary of paragraphs"""
    hits = pd.DataFrame([paras[para] for para in paras if word in paras[para]])
    return hits

In [100]:
def coll(word, paras, exp=1.2):
    if word in paras.index:
        res = (paras[[i for i in paras if paras[i][word] > 0]].sum(axis=1)**exp/freqs).sort_values(ascending=False)
    else:
        res = pd.Series()
    return res

In [101]:
def collwb(wordbag, paras, exp=1.2, column=None):
    res = pd.DataFrame()
    for w in wordbag:
        res[w] = coll(w, paras, exp)
    result = pd.DataFrame()
    if column == None:
        colname = 'coll'
    else:
        colname = column
    result[colname] = res.sum(axis=1)
    return result.sort_values(by=colname, ascending=False)

def combocoll(wordbag, paras, exps = None, freqs = None):
    
    if exps == None:
        exps = [1.05, 1.2]
    
    rescoll = pd.concat([collwb(wordbag, paras, exp = a, column = a) for a in exps], axis=1, sort=True)
    
    # wise to compute the frequencies beforehand, it takes a second or so off the time
    if freqs == None:
        freqs = paras.sum(axis=1)
    
    rescoll['freq'] = freqs[rescoll.index]
    
    return rescoll

In [102]:
def inspectcoll(collocation, freqlim = 0, colnum = None, colname = None):
    if colname == None:
        if colnum == None:
            column = collocation.columns[0]
        else:
            column = collocation.columns[colnum]
    else:
        column = colname
        
    return collocation[collocation.freq > freqlim].sort_values(by=column, ascending=False)

In [103]:
def displaycoll(collocation,n = 20, freqlim = 0):        
    for column in collocation:
        if column != 'freq':
            display(Markdown('### ' + str(column)))
            display(Markdown(', '.join(list(collocation[collocation.freq > freqlim].sort_values(by=column, ascending=False).index[:n]))))
            display(Markdown('<br>'))

In [104]:
wordbags.keys()

dict_keys(['Kvinde_', 'Sandhed_', 'Frihed_', 'Legeme_', 'Læserinde_', 'Litteratur_', 'Religion_', 'Kvindeforening_', 'Forfatterinde_', 'Myndighed_', 'Yrkesaktiv_', 'Sivil status_', 'Ligestilling_', 'Folkeoplysning_', 'Sundhed_', 'Befolkningsprosesser_'])

In [111]:
coll('kvinnor', texthot).head()

kvinnor       2.131526
vanligaste    1.245731
simmade       1.245731
lyssnen       1.148698
Låten         1.148698
dtype: float64

In [106]:
themes = dict()
for w in wordbags.keys():
    themes[w] = combocoll(wordbags[w], texthot, exps = [1.05, 1.2, 1.5])


# Her er kollokasjonene for temaordene

Passer greit nok på svensk, men noen er blanke.

In [108]:
for w in themes:
    display(Markdown("## "+ w))
    displaycoll(themes[w], n=40, freqlim=2)

## Kvinde_

### 1.05

husmoder, småbarnsskola, moderlig, tillsyn, moder, hustru, rådlösa, egenkärlek, grön, sorgliga, stött, mor, möta, damer, huldhet, dä, va, rensa, nedbrunna, självständig, döttrarnas, återkommo, värdighet, skämtade, upphov, övergick, Tro, olympiska, tinningarna, stav, Bed, vården, förskaffa, samtycka, berömde, företag, ungarna, spela, överlämna, språkade

<br>

### 1.2

...

befolkning, rensa, gränder, fjärdingen, Stora, herrskap, angår, värdighet, talaren, svetten, olympiska, förmögna, livstid, stolar, samkväm, samlat, tröttnade, handräckning, villervalla, ting, strumpor, förmiddagen, gator, elände, nedåt, värma, bodde, däråt, farlig, gammalt, flygande, sopfjärdingen, brunno, ordspråk, kristliga, prästen, torget, halva, förena, skaffa

<br>

### 1.5

befolkning, rensa, fjärdingen, gränder, Stora, herrskap, elände, samkväm, värdighet, förmögna, strumpor, svetten, talaren, angår, tröttnade, samlat, stolar, livstid, ting, handräckning, förmiddagen, gator, villervalla, olympiska, däråt, värma, flygande, farlig, gammalt, nedåt, bodde, stadens, Många, kristliga, fruntimmersförening, brunno, sopfjärdingen, förena, halva, börjat

<br>